# Simulação de Monte Carlo

Para mais informações: https://pbpython.com/monte-carlo.html

Esse exemplo tenta prever quanto de dinheiro uma empresa deve orçamentar para comissões de venda para o próximo ano. Esse exemplo é interessante de modelar, porque existe uma fórmula definida para calcular comissões.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

sns.set_style('whitegrid')

In [10]:
# Definição das variáveis
avg = 1 # Média
std_dev = .1 # Desvio padrão
num_reps = 500 # Representantes de vendas
num_simulations = 1000 # Número de simulações

# Gera uma lista percentagens que irá replicar nossa distribuição normal histórica
pct_to_target = np.random.normal(avg, std_dev, num_reps).round(2)

In [12]:
sales_target_values = [75_000, 100_000, 200_000, 300_000, 400_000, 500_000]
sales_target_prob = [.3, .3, .2, .1, .05, .05]
sales_target = np.random.choice(sales_target_values, num_reps, p=sales_target_prob)

In [13]:
df = pd.DataFrame(index=range(num_reps), data={'Pct_To_Target': pct_to_target,
                                               'Sales_Target': sales_target})

df['Sales'] = df['Pct_To_Target'] * df['Sales_Target']

In [14]:
def calc_commission_rate(x):
    """ Return the commission rate based on the table:
    0-90% = 2%
    91-99% = 3%
    >= 100 = 4%
    """
    if x <= .90:
        return .02
    if x <= .99:
        return .03
    else:
        return .04

In [15]:
df['Commission_Rate'] = df['Pct_To_Target'].apply(calc_commission_rate)
df['Commission_Amount'] = df['Commission_Rate'] * df['Sales']

In [16]:
# Define a list to keep all the results from each simulation that we want to analyze
all_stats = []

# Loop through many simulations
for i in range(num_simulations):

    # Choose random inputs for the sales targets and percent to target
    sales_target = np.random.choice(sales_target_values, num_reps, p=sales_target_prob)
    pct_to_target = np.random.normal(avg, std_dev, num_reps).round(2)

    # Build the dataframe based on the inputs and number of reps
    df = pd.DataFrame(index=range(num_reps), data={'Pct_To_Target': pct_to_target,
                                                   'Sales_Target': sales_target})

    # Back into the sales number using the percent to target rate
    df['Sales'] = df['Pct_To_Target'] * df['Sales_Target']

    # Determine the commissions rate and calculate it
    df['Commission_Rate'] = df['Pct_To_Target'].apply(calc_commission_rate)
    df['Commission_Amount'] = df['Commission_Rate'] * df['Sales']

    # We want to track sales,commission amounts and sales targets over all the simulations
    all_stats.append([df['Sales'].sum().round(0),
                      df['Commission_Amount'].sum().round(0),
                      df['Sales_Target'].sum().round(0)])

In [17]:
results_df = pd.DataFrame.from_records(all_stats, columns=['Sales',
                                                           'Commission_Amount',
                                                           'Sales_Target'])

In [19]:
results_df.describe().style.format('{:,}')

,Sales,Commission_Amount,Sales_Target
count,"1,000.0","1,000.0","1,000.0"
mean,"83,780,767.25","2,860,095.391","83,782,500.0"
std,"2,719,278.1423119125","102,889.568751599","2,688,598.1123644733"
min,"75,724,500.0","2,529,015.0","75,925,000.0"
25%,"81,898,562.5","2,789,791.0","81,887,500.0"
50%,"83,671,375.0","2,859,007.5","83,650,000.0"
75%,"85,750,187.5","2,930,399.0","85,725,000.0"
max,"92,049,500.0","3,221,992.0","91,900,000.0"


In [18]:
print(results_df)

          Sales  Commission_Amount  Sales_Target
0    86188750.0          2984870.0      86050000
1    87162000.0          3013792.0      86325000
2    85259500.0          2945495.0      84800000
3    87444500.0          2982300.0      87400000
4    82574250.0          2814385.0      82775000
5    83304750.0          2830080.0      83325000
6    85359750.0          2917545.0      85775000
7    83019000.0          2870158.0      82700000
8    85246500.0          2934185.0      85225000
9    83669500.0          2839652.0      83875000
10   82798000.0          2863262.0      82875000
11   85368750.0          2930902.0      85100000
12   84153250.0          2906958.0      84000000
13   81336250.0          2788105.0      80975000
14   86178500.0          2908562.0      86500000
15   85416750.0          2895852.0      85775000
16   85636750.0          2994895.0      85075000
17   81940500.0          2711928.0      82900000
18   83813500.0          2858780.0      84000000
19   87862000.0     